In [ ]:
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data
from sklearn.linear_model import LinearRegression
import sklearn

quality = ['manufacturer', 'model', 'transmission', 'color', 'engineFuel', 'engineType', 'bodyType', 'drivetrain']

def FormatCSV(data):
    data.iloc[:, :] = data.iloc[:, :].replace(to_replace = False, value = 0)
    data.iloc[:, :] = data.iloc[:, :].replace(to_replace = True, value = 1)
    
def Convert(data, quality):
    for x in quality:
        value = 1
        for y in data[x].unique():
            data[x] = data[x].replace(to_replace = y, value = value)
            value += 1
            

def Train(X, Y):
    print('Training...')
    linear_regressor = LinearRegression(n_jobs = -1)  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    
    print('Train completed!')
    return linear_regressor

def Predict(linear_regressor, X, Y):
    index = np.arange(0, len(X), 1)
    Y_pred = linear_regressor.predict(X)  # make predictions


    plt.scatter(index, Y)
    plt.plot(index, Y_pred, color='red')
    plt.show()

    rmse = sklearn.metrics.mean_squared_error(Y, Y_pred)**0.5
    print("R2:", linear_regressor.score(X, Y))
    print("RMSE:", rmse)
    
def Transform(dataset, resultset):
    print('Transform input data set...')
    data = pd.read_csv(dataset)
    result = pd.read_csv(resultset)

    data = data.sort_values(['id'])
    result = result.sort_values(['id'])

    data['price'] = result.price
    data = data.sort_values('price')
    result = result.sort_values('price')

    data = data.drop(['id'], axis = 1)
    data = data.dropna()

    Y = data.price.values

    FormatCSV(data)
    Convert(data, quality)

    #print( data.corr()['price'].sort_values(ascending = False)[1:])
    X = data.loc[:, data.corr().abs()['price'].sort_values(ascending = False)[1:].index]

    X_3 = X.loc[:, ['engineType', 'engineFuel', 'feature_7', 'feature_3', 'feature_8', 'feature_6', 'feature_5', 'feature_2', 'feature_4', 'feature_9', 'feature_1', 'feature_0']].values
    X_2 = X.loc[:, ['odometer', 'color']].values
    X_1 = X.loc[:, ['photos', 'bodyType', 'drivetrain', 'transmission', 'model', 'manufacturer', 'year', 'engineCapacity']].values

    #X_1 = np.nan_to_num(X_1)
    #X_2 = np.nan_to_num(X_2)

    #   'odometer', 'year','engineCapacity', 'photos', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9'
    #X = data.drop(['id', 'manufacturer', 'model', 'transmission', 'color', 'engineFuel', 'engineType', 'bodyType', 'drivetrain'], axis = 1).values



    #Y = result.drop(['id'], axis = 1).values

    from sklearn.preprocessing import PolynomialFeatures
    pr_1 = PolynomialFeatures(degree = 9, interaction_only = False)
    pr_2 = PolynomialFeatures(degree = 1, interaction_only = True)
    X_1 = pr_1.fit_transform(X_1)
    X_2 = pr_2.fit_transform(X_2)

    #coef = [4.77631076e+02, 1.18088513e+03, 7.49256638e+02, 2.03905728e+02, 1.20829463e+03, 1.02326110e+02 -3.28709490e-03, 6.63900844e+02, 5.11730154e+02, 2.26841810e+03, 2.94064985e+02, 7.47388250e+01, -1.07257328e+03, 7.90083064e+02]

    #Z = Transform(X, coef)
    Z = np.concatenate((X_1, X_2),axis=1)
    Z = np.concatenate((Z, X_3),axis=1)
    
    print('Transform completed!')
    
    return Z, Y

def GetData():
    dataset = input('X_train set: ')
    resultset = input('Y_train set: ')
    
    return dataset, resultset

dataset,resultset = GetData()

X, Y = Transform(dataset, resultset)

linear_regressor = Train(X, Y)

Predict(linear_regressor, X, Y)

X_train set: X_train.csv
Y_train set: Y_train.csv
Transform input data set...
Transform completed!
Training...
